## Предобработка изображений из `train` и `test` датасетов

**Цель:** 

1. _Аугментации_, так как для каждого класса их нужно разное количество `augs_need`

2. Изменение файловой структуры датасетов для удобного использования в `torch.Dataset`

In [1]:
import os
import numpy as np
import pandas as pd
import torch

from PIL import Image
from torchvision import transforms
import albumentations as A

from scipy.stats import bernoulli

import warnings
warnings.filterwarnings('ignore')

**Функция для загрузки и предобработки изображений**

In [2]:
# во сколько раз больше нужно объектов каждого класса для устранения дисбаланса классов
augs_need = np.array([2.47359155, 3.3452381 , 1.67261905, 2.31848185, 5.40384615,
                      2.91493776, 6.62735849, 2.87321063, 3.13616071, 5.87866109,
                      1.08076923, 1.        , 3.03455724, 1.35096154, 3.25986079,
                      1.13765182, 3.47772277, 3.4691358 , 1.02479942, 1.22280244,
                      3.37740385, 4.87847222, 1.29373849])


def load_images(data='train'):
    path = f"../input/dermnet/{data}/"
    list_cat = os.listdir(path)
    
    os.mkdir(f'{data}')
    os.mkdir(f'{data}/images')
    
    augmentation = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.GaussNoise(p=0.5, var_limit=(10, 30)), 
        A.RandomBrightnessContrast(p=0.5, brightness_limit=0.15, contrast_limit=0.15)
    ])
    
    idx = 0
    labels = torch.zeros(33000)
    for i, cat in enumerate(list_cat):      
        list_images = os.listdir(path + cat)
        aug_count = augs_need[i]

        for j in list_images:   
            with Image.open(path + cat + "/" + j) as input_image:
                for p in range(int(aug_count) + bernoulli.rvs(aug_count - int(aug_count))): 
                    # применяем аугментации
                    if p:
                        input_image = Image.fromarray(augmentation(image=np.array(input_image))["image"])
                    
                    # сохраняем изображения в новые директории
                    input_image.save(f'{data}/images/({idx})-img.jpg')    

                    # записываем лейблы в тензор
                    labels[idx] = i

                    # счётчик общего количества добавленных изображений
                    idx += 1
                    
                    if data == 'test':  # для теста аугментации не нужны
                        break

    labels.resize_(idx)
    torch.save(labels, f'{data}/labels.pt')
    

# загружаем датасеты
load_images('train')
load_images('test')